In [1]:
import pandas as pd
import numpy as np
from io import StringIO

Files imported: Students_export.csv, Student_race.csv <br>
File exported: eIEPPROStudentDemographics.csv

<H4> Things to do: <br>
* Validate students + RED & Hispanic Y/N data (largely done) <br>
* encapsulate all text in quotes (needs to be done) <br>
* Add student demographic info & Parent/Guardian contacts afterwards

<H5> StudentRace Table: converted to binary values 
    
| STUDENTRACE.StudentID |STUDENTRACE.RaceCd | ---|
|--- |--- |--- |
| Some# | Possibilities are: |
| AS | Asian | 
| WH | White | 
| BL | Black | 
| AM | American Native | 
| PI | Pacific Islander | 
| not in Student Race table | Hispanic |

Appendix 1 Spec (06/2022 Document v22)

| eIEPPRO Data Field | Powerschool Table Name | Details | 
| --- |--- |--- |
| StateIDNumber | STUDENTS.State_StudentNumber | nvarchar (10) |
| SISIDNumber   | STUDENTS.Student_Number      | nvarchar(9)   |
| SchoolCode    | STUDENTS.SchoolID            | nvarchar(9)   |  
| LastName      | STUDENTS.Last_Name           | nvarchar(40)  |
| FirstName     | STUDENTS.First_Name          | nvarchar(25)  |
| MiddleName    | STUDENTS.Middle_Name         | nvarchar(25)  |
| DOB           | STUDENTS.DOB                 | smalldatetime |
| Gender        | STUDENTS.Gender              | nvarchar(6)*  |  
| Ethnicity_HI  | STUDENTS.Hispanic Y/N        | Yes: HI == 1 |
| Ethnicity_AM  | STUDENTRACE.RaceCd == AM |--- |
| Ethnicity_AS  | STUDENTRACE.RaceCd == AS |--- |
| Ethnicity_BL  | STUDENTRACE.RaceCd == BL |--- |
| Ethnicity_PI  | STUDENTRACE.RaceCd == PI |--- |
| Ethnicity_WH  | STUDENTRACE.RaceCd == AM |--- |
| Grade | STUDENTS.Grade_Level |--- |
| Student Address #1          | STUDENTS.Mailing_Street     | Powerschool  ~ Street Address |
| Student Address #2          | Students.Mailing_City, Mailing_State and Mailing_Zip     |  |
| Student Physical Address ~ StudentPhyAddress  | STUDENTS.Street   | STUDENTS.Street |  
| Student Phone #             | StudentPhone        | STUDENTS.Home_Phone |
| Student E-Mail              | StudentEmail        | will not provide |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |
| PrimaryLanguage | S_AZ_STU_X.spokenlanguage |--- |
| HomeLanguage | S_AZ_STU_X.homelanguagecode |--- |
| District of Residence | DOR |--- |
| District of Attendance | DOA |--- |
| Home School | HomeSchool |--- |
| Attending School | AttendingSchool |--- |
| Emergency Contact | EmergencyContact |--- |
| Emergency Contact Phone | EmergencyContactPhone |--- |

* Gender: AzEDS defined Gender values <br>
* Students pulled based on ACTIVITIES.childfind504 == 1 or ACTIVITIES.childfindidea == 1 <br>
S_AZ_STU_X.firstacquiredlanguage <br>
S_AZ_STU_X.homelanguagecode is home language <br>
S_AZ_STU_X.spokenlanguage is the same as primary language <br>


<H3> Input Files: <br>

In [2]:
Students = pd.read_csv("Students_export.csv", encoding='latin-1')
StudentRaceAndEthnicity = pd.read_csv("StudentRace.csv", encoding='latin-1')

<H5> All students with the flags for childfind504 and childfindidea are pulled in. These flags are to be kept on for the time that students have a 504 or IEP. <br>
    
<H4> Consider running tests to compare Flag population to Population at Large - compare numbers

In [3]:
Students_504 = Students.loc[Students['ACTIVITIES.childfind504'] == 1] 
Students_and_idea = Students.loc[Students['ACTIVITIES.childfindidea'] == 1]

In [4]:
# 606 students
Students_504

,STUDENTS.SchoolID,STUDENTS.ID,STUDENTS.Student_Number,STUDENTS.Last_Name,STUDENTS.First_Name,STUDENTS.DOB,STUDENTS.Mother,STUDENTS.Father,STUDENTS.Grade_Level,STUDENTS.Home_Room,...,U_DEF_EXT_STUDENTS.sibling5hr,ELL Alert,ACTIVITIES.indicator1,ACTIVITIES.indicator2,ACTIVITIES.childfind504,ACTIVITIES.childfindidea,504 Alert,S_AZ_STU_X.firstacquiredlanguage,S_AZ_STU_X.homelanguagecode,S_AZ_STU_X.spokenlanguage
10,450,55,90001285,Jensen,Cash,2008-12-17,"Foshee, Heather","Jensen, Blake",8,"Ocasla-Johnson, Samantha",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
37,100,1500,20081142,Napier,Timothy,2009-03-08,"Napier, Rebecca","Napier, Timothy",8,"Brownfield, Cambria",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
80,350,2621,70001095,Dohaniuk,Jordan,2009-06-01,"Dohaniuk, Beth","Dohaniuk, Jeffrey",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
81,350,2623,70001183,Roberts,Katelyn,2009-01-09,"Roberts, Kimberly","Roberts, Jeffery",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
94,350,2655,70001083,Green,Cody,2008-12-18,"Green, Mindy","Green, Jason",8,"Spencer, Megan",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21115,650,221400,75982,Wyckoff,Amir,2016-08-11,"Registre, Mya","Wyckoff, James",1,"Seto, Aleksandra",...,.,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0,0.0
21151,500,221625,76032,Orrego,Benjamin,2013-07-08,"Orrego, Tiffany","Orrego, David",4,Hertko,...,.,NaN,NaN,NaN,1.0,NaN,Y,0.0,0,0.0
21207,800,222012,76096,Vodolazkyi,Rostyslav,2012-04-24,"Vodolazka, Nadiia","Vodolazkyi, Yurii",5,"Hinman, Douglas",...,.,1.0,NaN,NaN,1.0,NaN,NaN,22.0,22,22.0
21287,300,222422,76192,Truran,Mason,2008-10-17,"San Agustin-Rutty, Vanessa","Truran, Nicholas",8,"Palmaioli, Molly",...,.,NaN,NaN,NaN,1.0,NaN,Y,0.0,0,0.0


In [5]:
# 2022 students
Students_and_idea

,STUDENTS.SchoolID,STUDENTS.ID,STUDENTS.Student_Number,STUDENTS.Last_Name,STUDENTS.First_Name,STUDENTS.DOB,STUDENTS.Mother,STUDENTS.Father,STUDENTS.Grade_Level,STUDENTS.Home_Room,...,U_DEF_EXT_STUDENTS.sibling5hr,ELL Alert,ACTIVITIES.indicator1,ACTIVITIES.indicator2,ACTIVITIES.childfind504,ACTIVITIES.childfindidea,504 Alert,S_AZ_STU_X.firstacquiredlanguage,S_AZ_STU_X.homelanguagecode,S_AZ_STU_X.spokenlanguage
11,450,64,90001394,Laughlin,Isaac,2008-07-09,"Laughlin, Capri","Laughlin, Ryan",8,"Knoblock, Heidi",...,.,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,NaN
14,450,71,90001271,Mirand,Kirra,2008-11-28,"Mirand, Christina","Mirand, Nate",8,"Katz, Shannon",...,.,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,NaN
20,450,88,90001330,Reantazo,Zahara,2008-10-04,"Nieves, Catherine","Reantazo, Eduardo",8,"Ocasla-Johnson, Samantha",...,.,NaN,NaN,NaN,NaN,1.0,NaN,NaN,7,NaN
25,100,1444,20081071,Aguilera,Alberto,2008-12-06,"Aguilera, Perla","Aguilera, Alberto",8,"Rust, Jason",...,.,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1,NaN
34,100,1490,20081035,Olson,Bailey,2009-04-11,"Hunt, Megan","Olson, Steel",7,"Cullop, Dave",...,.,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21455,500,224977,76386,Farooqi,Christopher,2010-11-22,"Farooqi, Bella","Farooqi, Rizwan",6,Bradbury,...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
21461,1150,224983,76392,Davy,Adalynn,2012-12-29,"Davy, Kimberly",NaN,4,"Thomas, Debra",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
21465,1150,224987,76396,White,Jonathan,2009-03-06,"White, Sandra",NaN,7,"Richard, Jill",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
21476,1000,225011,76408,Hudson,Deshawn,2012-11-09,"Hudson, Shaqueila","Sherrill, Demylo",4,"Papamatheakis, Athena",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0


In [6]:
# Combine communities and drop duplicates
df_504_id = pd.concat([Students_504, Students_and_idea], ignore_index=True).drop_duplicates()

In [7]:
# 2616 combined
df_504_id

,STUDENTS.SchoolID,STUDENTS.ID,STUDENTS.Student_Number,STUDENTS.Last_Name,STUDENTS.First_Name,STUDENTS.DOB,STUDENTS.Mother,STUDENTS.Father,STUDENTS.Grade_Level,STUDENTS.Home_Room,...,U_DEF_EXT_STUDENTS.sibling5hr,ELL Alert,ACTIVITIES.indicator1,ACTIVITIES.indicator2,ACTIVITIES.childfind504,ACTIVITIES.childfindidea,504 Alert,S_AZ_STU_X.firstacquiredlanguage,S_AZ_STU_X.homelanguagecode,S_AZ_STU_X.spokenlanguage
0,450,55,90001285,Jensen,Cash,2008-12-17,"Foshee, Heather","Jensen, Blake",8,"Ocasla-Johnson, Samantha",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
1,100,1500,20081142,Napier,Timothy,2009-03-08,"Napier, Rebecca","Napier, Timothy",8,"Brownfield, Cambria",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
2,350,2621,70001095,Dohaniuk,Jordan,2009-06-01,"Dohaniuk, Beth","Dohaniuk, Jeffrey",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
3,350,2623,70001183,Roberts,Katelyn,2009-01-09,"Roberts, Kimberly","Roberts, Jeffery",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
4,350,2655,70001083,Green,Cody,2008-12-18,"Green, Mindy","Green, Jason",8,"Spencer, Megan",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,500,224977,76386,Farooqi,Christopher,2010-11-22,"Farooqi, Bella","Farooqi, Rizwan",6,Bradbury,...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2624,1150,224983,76392,Davy,Adalynn,2012-12-29,"Davy, Kimberly",NaN,4,"Thomas, Debra",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2625,1150,224987,76396,White,Jonathan,2009-03-06,"White, Sandra",NaN,7,"Richard, Jill",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2626,1000,225011,76408,Hudson,Deshawn,2012-11-09,"Hudson, Shaqueila","Sherrill, Demylo",4,"Papamatheakis, Athena",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0


In [ ]:
# Clean columns
df_504_id.columns = df_504_id.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('/', '')

In [ ]:
# Create the dataframe for writing, column by column
df_StudentData = df_504_id.filter(['students.state_studentnumber', 
                                  'students.student_number',
                                  'students.id',
                                  'students.schoolid',
                                  'students.last_name',
                                  'students.first_name',
                                  'students.middle_name',
                                  'students.dob',
                                  'students.gender',
                                  'hispanic_yn', #insert red data later                   
                                  'students.grade_level',
                                  'students.mailing_street',
                                  'students.mailing_city',
                                  'students.mailing_state',
                                  'students.mailing_zip',
                                  'students.street', # Using for physical street
                                  'students.home_phone', #Considering inserting contrived email
                                  's_az_stu_x.homelanguagecode',
                                  's_az_stu_x.spokenlanguage',
                                  'ell_alert',
                                  '504_alert'
                                 ])

In [ ]:
df_StudentData.set_index('students.student_number')

In [ ]:
df_StudentData

In [ ]:
df_StudentData.info()

<H5> Transforming Columns

In [ ]:
# Clean data as appropriate
# Turn State_StudentNumber to varchar
# df["Fee"]=df["Fee"].values.astype('string')
df_StudentData["stu_id"]        = df_StudentData["students.id"].values.astype('int')
df_StudentData["StateIDNumber"] = df_StudentData["students.state_studentnumber"].values.astype('int')
df_StudentData["SISIDNumber"]   = df_StudentData["students.student_number"].values.astype('int')
df_StudentData["SchoolCode"]    = df_StudentData["students.schoolid"].values.astype('int')
df_StudentData["LastName"]      = df_StudentData["students.last_name"]
df_StudentData["FirstName"]     = df_StudentData["students.first_name"]
df_StudentData["MiddleName"]    = df_StudentData["students.middle_name"]
df_StudentData["DOB"]           = pd.to_datetime(df_StudentData["students.dob"])
df_StudentData["Gender"]        = df_StudentData["students.gender"]
df_StudentData["Ethnicity_HI"]  = df_StudentData["hispanic_yn"]
# Break to insert RED Data

df_StudentData["Grade"]           = df_StudentData["students.grade_level"].values.astype('int')
df_StudentData["StudentAddress1"] = df_StudentData["students.mailing_street"]
df_StudentData["StudentAddress2"] = df_StudentData['students.mailing_city'].astype(str) + ', ' + df_StudentData['students.mailing_state'] + ', ' + df_StudentData['students.mailing_zip']
df_StudentData["StudentPhyAddress"] = df_StudentData["students.street"]
df_StudentData["StudentPhone"]      = df_StudentData["students.home_phone"]
df_StudentData["StudentEMail"]      = ""
# Insert Parent Contact Data

df_StudentData["PrimaryLanguage"]  = df_StudentData["s_az_stu_x.spokenlanguage"] #.values.astype('str') # clearing out int to see what happens
df_StudentData["HomeLanguage"]     = df_StudentData["s_az_stu_x.homelanguagecode"] # .values.astype('int')



<H5> Primary and Home Language Transformations:

In [ ]:
transform_language = {
    0:   'English', 
    1:   'Spanish', 
    2:   'Arabic', 
    3:   'Cambodian', 
    4:   'Cantonese', 
    5:   'Czechoslovakian', 
    6:   'Dutch', 
    7:   'Filipino', 
    8:   'Finnish', 
    9:   'French', 
    10:   'German', 
    11:   'Greek', 
    12:   'Hebrew', 
    13:   'Hungarian', 
    14:   'Italian', 
    15:   'Japanese', 
    16:   'Korean', 
    17:   'Laotian', 
    18:   'Mandarin', 
    19:   'Polish', 
    20:   'Portuguese', 
    21:   'Romanian', 
    22:   'Russian', 
    23:   'Thai', 
    24:   'Ukrainian', 
    25:   'Vietnamese', 
    26:   'Serbo-Croatian', 
    27:   'Other Non-Indian', 
    28:   'Navajo', 
    29:   'Apache (San Carlos)', 
    30:   'Apache (Whiteriver)', 
    31:   'Chemehuevi', 
    32:   'Cocopah', 
    33:   'Hopi', 
    34:   'Haulapai', 
    35:   'Kaibab-Paiute', 
    36:   'Maricopa', 
    37:   'Mohave', 
    38:   'Papago (Tohono Oodham)',
    39:   'Pima (Akimel Oodhman)',
    40:   'Supai', 
    41:   'Tewa', 
    42:   'Yaqui', 
    43:   'Yavapai', 
    44:   'Yuma', 
    45:   'Other Indian', 
    46:   'American Sign Language',
    47:   'Albanian', 
    48:   'Amharic', 
    49:   'Azeri', 
    50:   'Burmese', 
    51:   'Dari', 
    52:   'Dinka', 
    53:   'Farsi', 
    54:   'Kirundi', 
    55:   'Kru', 
    56:   'Lingala', 
    57:   'Maay', 
    58:   'Mandingo', 
    59:   'Nuer', 
    60:   'Pashto', 
    61:   'Serbian', 
    62:   'Somali', 
    63:   'Sawhili', 
    64:   'Tagalog', 
    65:   'Urdu', 
    66:   'Uzbek', 
    67:   'Vai', 
    68:   'Afrikaans', 
    69:   'Bengali', 
    70:   'Hindi', 
    71:   'Persian', 
    72:   'Punjabi', 
    73:   'Cherokee', 
    74:   'Armenian', 
    75:   'Bosnian', 
    76:   'Central Khmer', 
    77:   'Chuukese', 
    78:   'Gujarati', 
    79:   'Haitian; Haitian Creole', 
    80:   'Hmong; Mong', 
    81:   'Karen Languages', 
    82:   'Lao', 
    83:   'Malayalam', 
    84:   'Marshallese', 
    85:   'Nepali', 
    86:   'Otomian languages', 
    87:   'Samoan', 
    88:   'Tamil', 
    89:   'Telugu', 
    90:   'Tigrinya', 
    91:   'Turkish', 
    92:   'undeclared', 
    93:   'Twi', 
    94:   'Yupik languages'
}

df_StudentData["PrimaryLanguage"] = df_StudentData["PrimaryLanguage"].map(transform_language)
df_StudentData["HomeLanguage"] = df_StudentData["HomeLanguage"].map(transform_language)

<H5> District Transformations: <br>
District of Residence & Attendance: based on School ID, but LOA gets Surprise (double check)

In [ ]:
transform_schoolID = {
    100: 'Legacy Traditional School - Maricopa',
    150: 'Legacy Traditional School - Casa Grande',
    200: 'Legacy Traditional School - Queen Creek',
    250: 'Legacy Traditional School - Avondale',
    300: 'Legacy Traditional School - Chandler',
    350: 'Legacy Traditional School - NW Tucson',
    400: 'Legacy Traditional School - Laveen',
    450: 'Legacy Traditional School - Gilbert',
    500: 'Legacy Traditional School - Surprise',
    550: 'Legacy Traditional School - North Chandler',
    600: 'Legacy Traditional School - Glendale',
    650: 'Legacy Traditional School - Peoria',
    700: 'Legacy Traditional School - East Mesa',
    750: 'Legacy Traditional School - Phoenix',
    800: 'Legacy Traditional School - Goodyear',
    850: 'Legacy Traditional School - North Phoenix',
    900: 'Legacy Traditional School - West Surprise',
    950: 'Legacy Traditional School - East Tucson',
    1000: 'Legacy Traditional School - Deer Valley',
    1050: 'Legacy Traditional School - Mesa',
    1100: 'Legacy Online Academy', 
    1150: 'Legacy Traditional School - San Tan'
}

# This where LOA should get Surprise data
df_StudentData['DOR'] = df_StudentData['SchoolCode'].map(transform_schoolID)
df_StudentData['DOA'] = df_StudentData['SchoolCode'].map(transform_schoolID)

# Home School and Attending School his is where LOA stays LOA - plus students that are exceptions to the rule?
df_StudentData['HomeSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)
df_StudentData['AttendingSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)


In [ ]:
df_StudentData['EmergencyContact'] = ''
df_StudentData['EmergencyContactPhone'] = ''

<H4> Last Fields from Schools_Export - these students should have ChildFing Flags preserved: <br>
    * 504Student (comes from '504 Alert'), comes in as Y, translate to 1 <br> 
    * EL Student (comes from 'ELL Alert') <br>

In [ ]:
# sampleDF.housing.replace(('yes', 'no'), (1, 0), inplace=True)
df_StudentData["504Student"]       = df_StudentData['504_alert']
df_StudentData['504Student'].replace(('Y', ''), (1, 0), inplace=True)

In [ ]:
df_StudentData["ELStudent"]        = df_StudentData['ell_alert']

<H3> Student Race and Ethnicity data <br>
<H5> HI, AM, AS, BL, PI, WH must be converted to 1 = YES or 0 = No values, per student <br>
    based on STUDENTRACE.StudentID <br>
    STUDENTS.ID == STUDENTRACE.StudentID <br>

In [ ]:
# Create dataframe - this version works!
def BuildRaceMatrix8( df_StudentRace ):
    global df_RED
    df_RED = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_StudentRace.iterrows():
        student_ID = row['STUDENTRACE.StudentID']
        race_data = row['STUDENTRACE.RaceCd']
        df_RED.at[index, 'stu_id'] = student_ID
        
        match race_data:
            case 'AM':
                df_RED.at[index, 'AM'] = 1
            case 'AS':
                df_RED.at[index, 'AS'] = 1
            case 'BL':
                df_RED.at[index, 'BL'] = 1
            case 'PI':
                df_RED.at[index, 'PI'] = 1
            case 'WH':
                df_RED.at[index, 'WH'] = 1
    return df_RED

In [ ]:

testRed3 = BuildRaceMatrix8(StudentRaceAndEthnicity).fillna(0).astype(int)

In [ ]:
df_pivot = pd.pivot_table(testRed3, values=['stu_id', 'AS', 'WH', 'BL', 'AM', 'PI'], index=['stu_id'], aggfunc=np.sum, margins=True)

In [ ]:
df_Pivot_plus_StudentData = df_StudentData.join(df_pivot, on='stu_id')

In [ ]:
df_Pivot_plus_StudentData.info()

In [ ]:
, ['SISIDNumber'], ['SchoolCode'], ['LastName'], ['FirstName'], ['MiddleName']
 

In [ ]:
df_write = df_Pivot_plus_StudentData[['StateIDNumber', 'SISIDNumber', 'SchoolCode', 'LastName', 'FirstName', 'MiddleName',
                                      'DOB','Gender', 'Ethnicity_HI', 'AM', 'AS', 'BL', 'PI', 'WH', 'Grade', 'StudentAddress1',
                                      'StudentAddress2', 'StudentPhyAddress', 'StudentPhone', 'StudentEMail', 'PrimaryLanguage',
                                      'HomeLanguage', 'DOR', 'DOA', 'HomeSchool', 'AttendingSchool', 'EmergencyContact', 
                                      'EmergencyContactPhone', '504Student', 'ELStudent'
                                     ]]

In [ ]:
df_write.info()

<H1> Lastly, Encapsulate Strings with quotes, Fill in NaN values with 0

In [ ]:
df_write.to_csv('IEPPRO_Draft_forUpload.csv', encoding='utf-8')